In [ ]:
import pandas as pd
import os

# File Paths
data_location = "/Users/nattybatty/HPC_Emissions_project/emissions-tools/raw_data/"
embodied_carbon_path = os.path.join(data_location, "node-models-embodiedcarbon (1).xlsx")
node_info_path = os.path.join(data_location, "node-info-combined-2025-04-01.xlsx")

embodied_df = pd.read_excel(embodied_carbon_path)
node_info_df = pd.read_excel(node_info_path)

# Clean embodied data
embodied_df = embodied_df.rename(columns={
    "Model": "Model",
    "Mean Emissinos (Kg CO2e)": "embodied_carbon_kgco2e"
})
embodied_mean = embodied_df.drop_duplicates(subset=["Model"])[["Model", "embodied_carbon_kgco2e"]]

# Clean Node Data
node_info_df = node_info_df.rename(columns={
    "Name": "node_name",
    "CPUs": "cpu_cores",
    "Model": "Model",
    "Manufacturer": "Manufacturer"
})

# Create a new column for model_manufacturer_id
node_info_df["model_manufacturer_id"] = (
    node_info_df["Manufacturer"].str.strip().str.replace(" ", "") + "_" +
    node_info_df["Model"].str.replace(" ", "")
)

# Normalize invalid strings to NaN (fixing issue)
node_info_df.replace(["nan", "NaN", "N/a"], pd.NA, inplace=True)

# Convert to numeric safely
node_info_df["cpu_cores"] = pd.to_numeric(node_info_df["cpu_cores"], errors="coerce")

# Select required columns
node_info_df = node_info_df[["node_name", "cpu_cores", "Model", "model_manufacturer_id"]]

# Merge & Fill 
merged_df = pd.merge(node_info_df, embodied_mean, on="Model", how="left")

# Keep original NaNs in one column, fill average in another
avg_embodied = pd.to_numeric(merged_df["embodied_carbon_kgco2e"], errors="coerce").mean()
merged_df["embodied_carbon_filled"] = merged_df["embodied_carbon_kgco2e"].fillna(avg_embodied)

# Add energy usage
merged_df["energy_100pct_kw"] = 0.41
merged_df["energy_0pct_kw"] = 0.20

merged_df.head(20)


,node_name,cpu_cores,Model,model_manufacturer_id,embodied_carbon_kgco2e,embodied_carbon_filled,energy_100pct_kw,energy_0pct_kw
0,ernmesclearml01,NaN,ThinkSystem SR670 V2,Lenovo_ThinkSystemSR670V2,12000.0,12000.000000,0.41,0.2
1,erchpccomptest1,NaN,ThinkSystem SR645,Lenovo_ThinkSystemSR645,13036.0,13036.000000,0.41,0.2
2,erc-tre-hpc-comp005,NaN,ThinkSystem SR675 V3,Lenovo_ThinkSystemSR675V3,NaN,10343.072785,0.41,0.2
3,erc-tre-hpc-comp004,NaN,ThinkSystem SR675 V3,Lenovo_ThinkSystemSR675V3,NaN,10343.072785,0.41,0.2
4,erc-tre-hpc-comp003,NaN,ThinkSystem SR670 V2,Lenovo_ThinkSystemSR670V2,12000.0,12000.000000,0.41,0.2
5,erc-hpc-comp227,256.0,ThinkSystem SR645,Lenovo_ThinkSystemSR645,13036.0,13036.000000,0.41,0.2
6,erc-hpc-comp226,256.0,ThinkSystem SR645,Lenovo_ThinkSystemSR645,13036.0,13036.000000,0.41,0.2
7,erc-hpc-comp225,256.0,ThinkSystem SR645,Lenovo_ThinkSystemSR645,13036.0,13036.000000,0.41,0.2
8,erc-hpc-comp224,256.0,ThinkSystem SR645,Lenovo_ThinkSystemSR645,13036.0,13036.000000,0.41,0.2
9,erc-hpc-comp223,256.0,ThinkSystem SR675 V3,Lenovo_ThinkSystemSR675V3,NaN,10343.072785,0.41,0.2


In [72]:
# Save CSVs 

# 1. Keep original embodied values ( include NaNs)
final_df_actual = merged_df[[
    "node_name", "cpu_cores", "embodied_carbon_kgco2e",
    "energy_100pct_kw", "energy_0pct_kw", "model_manufacturer_id"
]].sort_values("node_name")
final_df_actual.to_csv("cleaned_nodes_actual_embodied.csv", index=False)

# 2. Replace NaNs with mean for safe usage downstream
final_df_filled = merged_df[[
    "node_name", "cpu_cores", "embodied_carbon_filled",
    "energy_100pct_kw", "energy_0pct_kw", "model_manufacturer_id"
]].rename(columns={"embodied_carbon_filled": "embodied_carbon_kgco2e"}).sort_values("node_name")
final_df_filled.to_csv("cleaned_nodes_filled_embodied.csv", index=False)



In [73]:
import pandas as pd

# Load the filled (average-backed) data
df = pd.read_csv("cleaned_nodes_filled_embodied.csv")

# Reference mean used for filling
ESTIMATE_VALUE = 10343.0727848101

# Build node carbon dictionary
node_carbon = {
    row["node_name"]: {
        "cpu_cores": row["cpu_cores"],
        "embodied": row["embodied_carbon_kgco2e"],
        "embodied_estimate": round(row["embodied_carbon_kgco2e"], 6) == round(ESTIMATE_VALUE, 6),
        "max_power_draw": row["energy_100pct_kw"],
        "min_power_draw": row["energy_0pct_kw"],
        "model_id": row["model_manufacturer_id"]
    }
    for _, row in df.iterrows()
}


print(node_carbon["erc-hpc-comp131"])


{'cpu_cores': 32.0, 'embodied': 7730.0, 'embodied_estimate': False, 'max_power_draw': 0.41, 'min_power_draw': 0.2, 'model_id': 'Dell_PowerEdgeR640'}
